In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')
output=model.invoke("hi")
print(output.content)

In [40]:


def function1(state: AgentState) -> AgentState:
    state["output"] = state["input"] + " from first function"
    state["step"] = "model"  # optional, if you're using it for control flow
    return state

In [52]:
def function2(state: AgentState) -> AgentState:
    state["output"] = state["output"] + " Pudur from second function"
    state["step"] = "end"  # optional
    return state

In [54]:
from langgraph.graph import StateGraph
from typing import TypedDict, Literal


In [55]:
class AgentState(TypedDict):
          # The input
          input: str
          # The generated output
          output: str
          # The current step (used for control flow)
# Create the graph
workflow1 = StateGraph(AgentState)

In [56]:
workflow1.add_node("fun1",function1)


In [57]:
workflow1.add_node("fun2",function2)


In [58]:
workflow1.add_edge("fun1","fun2")

In [59]:

workflow1.set_entry_point("fun1")

In [60]:

workflow1.set_finish_point("fun2")

In [61]:
app=workflow1.compile()

In [ ]:
app.get_graph()


In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))


In [62]:
app.invoke({"input":"vivek"})

{'input': 'vivek',
 'output': 'vivek from first function Pudur from second function'}

First function as LLM and second function as Token counter


In [63]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        ("system", "{context}"),
        ("user", "{question}"),
    ]
)
prompt

model = ChatGroq(model="gemma2-9b-it",temperature=0.0, max_tokens=5000)


In [70]:
model.invoke("what is the capital of India?").content

'The capital of India is **New Delhi**. \n'

In [71]:
from typing import TypedDict, Literal

class AgentState(TypedDict):
    input: str
    output: str
    step: Literal["enter", "model", "count_tokens", "end"]
    messages: list
    token_count: int


In [72]:
def llm_function(state: AgentState) -> AgentState:
    result = model.invoke(state["input"])
    state["output"] = result.content
    state["messages"].append({"role": "model", "content": result.content})
    state["step"] = "count_tokens"
    return state


In [73]:
def token_counter_function(state: AgentState) -> AgentState:
    # Very basic token counter – can be replaced with tiktoken or tokenizer
    token_count = len(state["output"].split())
    state["token_count"] = token_count
    state["step"] = "end"
    return state


In [74]:
from langgraph.graph import StateGraph

graph = StateGraph(AgentState)

graph.add_node("llm_node", llm_function)
graph.add_node("token_counter", token_counter_function)

graph.add_edge("llm_node", "token_counter")

graph.set_entry_point("llm_node")
graph.set_finish_point("token_counter")

app = graph.compile()


In [76]:
initial_state = {
    "input": "Explain what LangGraph is in very detailed terms.",
    "output": "",
    "step": "enter",
    "messages": [],
    "token_count": 0
}

result = app.invoke(initial_state)
print("LLM Output:", result["output"])
print("Token Count:", result["token_count"])


LLM Output: ## LangGraph: A Deep Dive into the World of Language Representation

LangGraph is a powerful and innovative approach to representing language, moving beyond traditional word embeddings and venturing into a more nuanced understanding of linguistic relationships. 

**At its core, LangGraph constructs a graph where:**

* **Nodes:** Represent individual words or subword units (like morphemes or characters).
* **Edges:** Capture semantic and syntactic relationships between these words.

This graph-based representation allows LangGraph to capture complex linguistic phenomena that are often overlooked by simpler embedding models.

**Here's a detailed breakdown of LangGraph's key features and functionalities:**

**1. Graph Construction:**

* **Word Embeddings as Node Features:** LangGraph starts with pre-trained word embeddings (like Word2Vec or GloVe) to initialize the nodes with semantic information.
* **Relationship Extraction:**  It then utilizes various techniques to extract r